In [1]:
import pandas as pd

input_csv = "/Users/inesschwartz/Desktop/model/model_datasets/test_data.csv"
test_df = pd.read_csv(input_csv)

print(test_df.shape)
test_df.head()

(184, 53)


,log_soc_stock,DEM,MRRTF,MRVBF,TWI,X_coord,Y_coord,aspect,bio1,bio12,...,slope,slope_height,slope_length,standardized_height,terrain_surf_texture,terrain_surv_conv,tmax,total_curve,valley_depth,year
0,1.11,1477.82140,6.960664,8.731725,21.366892,589028.466030,8.656649e+06,3.976567,199.0,1268.0,...,0.006761,104.79858,855.36206,579.76710,58.423077,44.448780,26.845642,1.332990e-10,393.32556,1956.0
1,1.62,1201.55080,7.354778,8.700499,20.198360,945265.031883,8.838035e+06,2.677685,213.0,1352.0,...,0.014329,70.60196,320.81927,549.03900,67.006325,52.374790,27.500935,2.651238e-10,209.56810,1965.0
2,1.44,435.82562,4.116148,8.915597,24.163515,406734.521508,9.246797e+06,2.093850,242.0,949.0,...,0.010341,82.65298,10737.68300,164.72330,44.504020,48.280210,28.596115,1.972400e-10,657.34130,1959.0
3,2.07,1597.73490,6.973146,8.745792,23.793827,564265.548316,8.534381e+06,3.633138,190.0,1203.0,...,0.021065,110.13017,2083.10550,859.42270,60.393772,48.716830,25.723263,1.016203e-10,235.15665,1946.0
4,1.70,697.45070,3.153459,8.850451,20.343992,699269.488876,9.072847e+06,4.915011,238.0,1203.0,...,0.007255,93.91217,0.00000,230.60329,44.718365,44.274822,31.040636,1.172393e-10,597.87370,1963.0


In [9]:
import os
import rasterio
import geopandas as gpd
import numpy as np

# --------------------------
# Path to RFOK residual kriging SOC prediction raster
# --------------------------
rfok_res_raster = "/Users/inesschwartz/Desktop/model/bagged_maps_RFOK/RFOKres_SOC_mean_MgCha1.tif"

if not os.path.exists(rfok_res_raster):
    raise FileNotFoundError(f"❌ Raster not found: {rfok_res_raster}")

## rfok res raster
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# Convert test_df to GeoDataFrame
gdf_test = gpd.GeoDataFrame(
    test_df,
    geometry=gpd.points_from_xy(test_df.X_coord, test_df.Y_coord),
    crs="EPSG:32733"
)

# Open raster
with rasterio.open(rfok_res_raster) as src:
    gdf_test["rf_pred"] = [
        val[0] if val[0] != src.nodata else np.nan
        for val in src.sample([(geom.x, geom.y) for geom in gdf_test.geometry])
    ]


In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ## in log
y_true = gdf_test["log_soc_stock"].values

# Back-transform test observations (ln → original units)
y_true = np.exp(gdf_test["log_soc_stock"].values)

# Predictions already in Mg C/ha
y_pred = gdf_test["rf_pred"].values

# Remove NA rows
mask = np.isfinite(y_true) & np.isfinite(y_pred)
y_true = y_true[mask]
y_pred = y_pred[mask]

# Compute metrics
mse = mean_squared_error(y_true, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
bias = np.mean(y_pred - y_true)

bias = np.mean(y_pred - y_true)

print("\n🔍 Validation metrics against Test 1:")
print(f"R²   : {r2:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"MAE  : {mae:.3f}")
print(f"Bias : {bias:.3f}")



🔍 Validation metrics against Test 1:
R²   : 0.309
RMSE : 2.088
MAE  : 1.249
Bias : -0.227


In [11]:
print("y_true range:", np.nanmin(y_true), np.nanmax(y_true))
print("y_pred range:", np.nanmin(y_pred), np.nanmax(y_pred))
print("Mean y_true:", np.nanmean(y_true))
print("Mean y_pred:", np.nanmean(y_pred))


y_true range: 1.0 20.6972325893895
y_pred range: 1.3149174 8.713351
Mean y_true: 4.410617341464961
Mean y_pred: 4.1834745


In [5]:
np.nanmean(np.log(y_pred))


0.29430267